In [ ]:
import requests
import json
import hmac
import hashlib
import base64
from urllib.parse import urlencode
import time
import uuid
import datetime
import threading
from websocket import create_connection
from ta_calculat import runmain
import pandas as pd
import re
import chime
import traceback
import numpy as np
import socket

def lot_size():
        url = " https://api-futures.kucoin.com/api/v1/contracts/active"
        payload={}
        files={}
        headers = {}

        pairs = requests.request("GET", url, headers=headers, data=payload, files=files)
        pairs=pairs.json()
        pairs=pairs['data']
        pairs=pd.DataFrame(pairs)
        pairs.drop(pairs.loc[pairs['quoteCurrency']!="USDT"].index, inplace=True)
        pairs.rename(columns={"symbol":"pair"},inplace=True)
        pairs=pairs[['pair','multiplier']]
        pairs=pd.DataFrame(pairs)
        pairs.set_index('pair', inplace=True)
        json.dump(pairs.to_dict('dict'), open('lot_size.json', 'w'))
        return 
lot_size()

class trader():

    def __init__(self,rond=None,commission=None,username=None):

        self.detail_accuonts ={}
        self.price =0
        self.rond =rond
        self.runpricetr =None
        self.ws =0
        self.alive_positions ={}
        self.commission =commission
        self.touched_sl =0
        self.sl_tp ={'stop_trail':0,'stopPrice_sl':0}
        self.base_uri = 'https://api-futures.kucoin.com'

        self.touched_st =0
        self.one_lot=None
        self.api_correct=[]
        self.username=username
        self.socket_message=None
        self.socket_response=None
        self.exit_trader=1
        self.symbol='XRPUSDTM'
        
    def creat_accuont(self):
        def accuo():

            self.base_uri = 'https://api-futures.kucoin.com'
            try:
                user=json.load(open('users.json'))
            except Exception as error:
                pass
            
            ind=['availableBalance','calculated_size','type_ballance','api_key','api_passphrase','stop_trail',
                 'api_secret','id','leverage','side','size_position','symbol','status','stopPrice_sl','lastprice',
                 'trailprice','actionprice','sl','st','status_sl','orderId_position','orderId_sl','last_time',
                 'username_trader','PNL','balance_trade']
            column=[]
            for i in [*user]:
                if (user[i]['username_trader']==self.username) and (user[i]['bot_code']==1):
                    column.append(i)
            
            if len(column)>0:
                ind_key = {key: 0 for key in ind}
                self.detail_accuonts = {key: ind_key for key in column}

                for i in column:
                    self.detail_accuonts[i]['api_key']=user[i]['api_key']
                    self.detail_accuonts[i]['api_secret']=user[i]['api_secret']
                    self.detail_accuonts[i]['api_passphrase']=user[i]['api_passphrase']
                    self.detail_accuonts[i]['status']=0
                    self.detail_accuonts[i]['status_sl']=0
                    self.detail_accuonts[i]['type_ballance']=['XBT','USDT','XRP','ETH','DOT']
                    self.detail_accuonts[i]['last_time']=str(datetime.datetime.utcnow())
                    self.detail_accuonts[i]['balance_trade']=user[i]['balance_trade']
                    
            else:
                self.exit_trader=0
            
            return

        try:
            self.detail_accuonts=json.load(open('detail_accuonts.json' ))
            print('exist position')
            self.symbol=self.detail_accuonts[self.username]["symbol"]
            self.multi_threading(func=self.check_exist_alive_position,accuonts=[*self.detail_accuonts])
            check=0
            for i in [*self.detail_accuonts]:
                if self.detail_accuonts[i]['status']>0:
                    check=1
                    break
            if check==0:
                accuo()
        except Exception as error:
            print(error)
            accuo()
            self.multi_threading(func=self.check_exist_alive_position,accuonts=[*self.detail_accuonts])
        if len([*self.detail_accuonts])>0:

            print(self.detail_accuonts)
            json.dump(self.detail_accuonts, open('detail_accuonts.json' , 'w'))
        return self.detail_accuonts
    
    def connection(self):

        x=1
        host = '127.0.0.1'
        port = 10000

        ClientSocket = socket.socket()
        print('Waiting for connection')
        try:
            ClientSocket.connect((host, port))
        except socket.error as e:
            print(str(e))
        Response = ClientSocket.recv(2048)
        while x==1:
            if self.socket_message!=None:
                self.socket_message=None
                ClientSocket.send(str.encode(Input))
            self.socket_response = ClientSocket.recv(2048).decode('utf-8')
        ClientSocket.close()

    def runprice(self):

        url='https://api.kucoin.com/api/v1/bullet-public'
        token=json.loads(requests.post(url).text)['data']['token']
        self.ws=create_connection('wss://ws-api.kucoin.com/endpoint?token=%s&[connectId=%s]' % (token,uuid.uuid1()))

        msg = {"id":"%s" % (uuid.uuid1()),
            "type": "subscribe",
            "topic": "/contractMarket/ticker:%s" %(self.symbol),
            "response": True}
        msg=json.dumps(msg)
        self.ws.send(msg)
        i=0
        self.runpricetr=1
        while self.runpricetr==1:
            if self.exit_trader==0:
                self.runpricetr=0
            try:
                p = json.loads(self.ws.recv())#
                if "data" in p:
                    if 'price' in p['data']:
                        self.price=p['data']['price']
    #                     rond=len(str(price).split(".")[1])
#                         print(self.price,end='-')

                if i==3:
                    msg = {"id":"%s" % (uuid.uuid1()),
                "type":"pong"
                          }
                    msg=json.dumps(msg)
                    self.ws.send(msg)
                    i=0
                i+=1
            except Exception as error:
    #             print('break connection with websocket')
#                 print('.',end='-')
                thread=threading.Thread(target=self.runprice)
                thread.start()
                break

    def refresh_runprice(self) :
        
        pr=0
        prr=1
        x=1
        while x==1:
            if self.exit_trader==0:
                x=0
            pr=self.price    
            time.sleep(5)
            prr=self.price
            if pr==prr:
                self.ws.close()
                self.runpricetr=0
                thread=threading.Thread(target=self.runprice)
                thread.start()
        return

    def set_param_orders(self,leverage,side,size,stopPrice_sl=None,stopPrice_tp=None,maintrade=True,
                         sl_trade=True,tp_trade=True,close_position=False):
        listparams=[]  #   for main position  listparams=['main position','stop los', 'take profit']
        if maintrade==True:
            params ={'clientOid': str(uuid.uuid1()),
                    'side': side,
                    'symbol': self.symbol,
                    'type': "market",
                    'size': str(size),
                    'leverage': str(leverage),
                    'remark':'maintrade'}
            data_json = json.dumps(params)
            listparams.append(data_json)

        if side=='buy':

            if sl_trade==True:
                params ={'symbol':self.symbol,
                        'side':'sell',
                        'leverage':str(leverage),
                        'type': 'market',
                        'clientOid':str(uuid.uuid1()),
                        'size':str(size),
                        'stop':'down',
                        'stopPrice':str(stopPrice_sl),
                        'stopPriceType':'MP',
                        'closeOrder':'true'}
                data_json = json.dumps(params)
                listparams.append(data_json)

            if tp_trade==True:
                params ={'symbol':self.symbol,
                        'side':'sell',
                        'leverage':str(leverage),
                        'type': 'market',
                        'clientOid':str(uuid.uuid1()),
                        'size':str(size),
                        'stop':'up',
                        'stopPrice':str(stopPrice_tp),
                        'stopPriceType':'MP',
                        'closeOrder':'true'}
                data_json = json.dumps(params)
                listparams.append(data_json)

            if close_position==True:
                params ={'symbol':self.symbol,
                        'side':'sell',
                        'leverage':str(leverage),
                        'type': 'market',
                        'clientOid':str(uuid.uuid1()),
                        'size':str(size),
                        'stopPriceType':'MP',
                        'closeOrder':'true'}
                data_json = json.dumps(params)
                listparams.append(data_json)

        elif side=='sell':

            if sl_trade==True:
                params ={'symbol':self.symbol,
                        'side':'buy',
                        'leverage':str(leverage),
                        'type': 'market',
                        'clientOid':str(uuid.uuid1()),
                        'size':str(size),
                        'stop':'up',
                        'stopPrice':str(stopPrice_sl),
                        'stopPriceType':'MP',
                        'closeOrder':'true'}
                data_json = json.dumps(params)
                listparams.append(data_json)

            if tp_trade==True:
                params ={'symbol':self.symbol,
                        'side':'sell',
                        'leverage':str(leverage),
                        'type': 'market',
                        'clientOid':str(uuid.uuid1()),
                        'size':str(size),
                        'stop':'down',
                        'stopPrice':str(stopPrice_tp),
                        'stopPriceType':'MP',
                        'closeOrder':'true'}
                data_json = json.dumps(params)
                listparams.append(data_json)

            if close_position==True:
                params ={'symbol':self.symbol,
                        'side':'buy',
                        'leverage':str(leverage),
                        'type': 'market',
                        'clientOid':str(uuid.uuid1()),
                        'size':str(size),
                        'stopPriceType':'MP',
                        'closeOrder':'true'}
                data_json = json.dumps(params)
                listparams.append(data_json)

        return listparams

    def get_headers(self,method,endpoint,parameters,user=None):
        api_key=self.detail_accuonts[user]['api_key']
        api_secret=self.detail_accuonts[user]['api_secret']
        api_passphrase=self.detail_accuonts[user]['api_passphrase']
        now = int(time.time() * 1000)
        if parameters=='':
            str_to_sign = str(now) + method + endpoint
        else:
            str_to_sign = str(now) + method + endpoint + parameters
        signature = base64.b64encode(hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest()).decode()
        passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest()).decode()
        return {'KC-API-SIGN': signature,
                'KC-API-TIMESTAMP': str(now),
                'KC-API-KEY': api_key,
                'KC-API-PASSPHRASE': passphrase,
                'KC-API-KEY-VERSION': '2',
                'Content-Type': 'application/json'}

    def request_order(self,meth,num,addendpoint,parameters=None,user=None):
        getmethod={'GET':{'1':'account-overview?currency=','2':'transaction-history','3':'deposit-address','4':'deposit-list',
        '5':'withdrawals/quotas','6':'withdrawal-list','7':'orders?status=done','8':'stopOrders?symbol=','9':'recentDoneOrders',
        '10':'fills','11':'recentFills','12':'openOrderStatistics','13':'positions?symbol=','14':'position?symbol=',
        '15':'contracts/risk-limit/','16':'funding-history?symbol=','method':'GET'},

        'POST':{'1':'orders','2':'position/margin/auto-deposit-status','3':'position/margin/deposit-margin',
        '4':'position/risk-limit-level/change','method':'POST'},

        'DELETE':{'1':'orders/','2':'orders?symbol=','3':'stopOrders?symbol=','method':'DELETE' }}

        method = getmethod[meth]['method']
        if addendpoint=='':
            endpoint = '/api/v1/%s' % (getmethod[method][num])
        else:
            endpoint = '/api/v1/%s' % (getmethod[method][num]) + addendpoint
            print(endpoint)
        re=''
        x=1
        while x==1:
            try:
                response = requests.request(method, self.base_uri+endpoint,
                    headers=self.get_headers(method,endpoint,parameters,user=user), data=parameters)

                print(response.status_code)
                if response.status_code==200:
                    if user in self.api_correct:
                        self.api_correct.remove(user)
                    break
                elif response.status_code==300003:
                    print('no enough money')
                    break
                elif response.status_code==429:
                    print('Too Many Requests')
                elif response.status_code==200002:
                    print('time sleep 10')
                    time.sleep(11)
                elif (response.status_code==400001) and (response.status_code==400002) and (response.status_code==400003)\
                and (response.status_code==400004):
                    self.api_correct.append(user)
                    print('self.api_correct=0 for trader %s and user %s' % (self.username , user))
                    break
                 
                elif response.status_code!=429 or response.status_code!=200002 :
                    print(response.status_code ,' : ', response.json()['msg'])
                    break
            except Exception as error:
                re='request_order'
                print(re)
                print(error)
                break
            
        return response.json() , re , response.status_code

    def open_position(self,user,leverage,side,stopPrice_sl):
        try:
            userp=json.load(open('users.json'))
        except Exception as error:
            pass
        self.detail_accuonts[user]['balance_trade']=userp[user]['balance_trade']
        cost=leverage*(self.detail_accuonts[user]['availableBalance']*(self.detail_accuonts[user]['balance_trade']/100))
        size=int(cost/(self.one_lot*self.price))
        self.detail_accuonts[user]['calculated_size']=size
        no_enough_money=1
        if size<1:
            no_enough_money=0

        if (self.detail_accuonts[user]['status']==0) and (no_enough_money==1) and (user not in self.api_correct):
            if side=='sell':
                print('sell-open_position')
                listparams=self.set_param_orders(leverage=leverage,side=side,size=size,
                stopPrice_sl=stopPrice_sl,tp_trade=False)
                for parameters in listparams:
                    response,re,statusCode=self.request_order(num='1',parameters=parameters,meth='POST',addendpoint='',user=user)
                    if listparams.index(parameters)==0:
                        if statusCode==200:
                            self.detail_accuonts[user]['status']=1
                            self.detail_accuonts[user]['orderId_position']=response['data']['orderId']
                        else:
                            break
                    if listparams.index(parameters)==1:
                        if statusCode==200:
                            self.detail_accuonts[user]['status_sl']=1
                            self.detail_accuonts[user]['orderId_sl']=response['data']['orderId']

            if side=='buy':
                print('buy-open_position')
                listparams=self.set_param_orders(leverage=leverage,side=side,size=size,
                stopPrice_sl=stopPrice_sl,tp_trade=False)
                for parameters in listparams:
                    response , re , statusCode=self.request_order(parameters=parameters,meth='POST',num='1',addendpoint='',user=user)
                    if listparams.index(parameters)==0:
                        if statusCode==200:
                            self.detail_accuonts[user]['status']=1
                            self.detail_accuonts[user]['orderId_position']=response['data']['orderId']
                        else:
                            break
                    if listparams.index(parameters)==1:
                        if statusCode==200:
                            self.detail_accuonts[user]['status_sl']=1
                            self.detail_accuonts[user]['orderId_sl']=response['data']['orderId']

        if self.detail_accuonts[user]['status']==1:
            self.detail_accuonts[user]['side']=side
            self.detail_accuonts[user]['size_position']=size
            self.detail_accuonts[user]['leverage']=leverage
            self.detail_accuonts[user]['symbol']=self.symbol
            x=1
            i=0
            while x==1:
    #             try:
                response,re,statusCode=self.request_order(parameters='',meth='GET',num='13',addendpoint=self.symbol,user=user)
                if statusCode==200 and len(response['data'])!=0:  # این باید 200 بشه و باید اینقدر تلاش کنه که جواب بگیره مگرنه باید پوزیشن را ببنده و اینو باید بعد از باز کردن پوزیشن تو قسمت اپن اردر بزارم و چک کنم که اونهایی که  اکشن پرایس و تریل پرایس ندارن بسته بشن یعنی یک تابع مخصوص کلوز بزارم
                    entryprice=response['data'][0]['avgEntryPrice']
                    self.detail_accuonts[user]['id']=response['data'][0]['id']
                    self.detail_accuonts[user]['trailprice']=entryprice
                    self.detail_accuonts[user]['actionprice']=entryprice
                    break
                if  len(response['data'])==0:
                    i+=1
                if i==2:
                    entryprice=self.price
                    self.detail_accuonts[user]['trailprice']=entryprice
                    self.detail_accuonts[user]['actionprice']=entryprice
                    break
        json.dump(self.detail_accuonts, open('detail_accuonts.json' , 'w'))
        
        return

    def control_sl(self,user,leverage,side):
        

        if self.detail_accuonts[user]['status']==1:
            parameters=''
            x=1
            c=0
            size=self.detail_accuonts[user]['calculated_size']
            if side=='buy':
                print('buy-control_sl')
                while x==1:
                    response,re,statusCode=self.request_order(parameters='',meth='GET',num='13',addendpoint=self.symbol,user=user)
                    if c>=1:
                        listparams=self.set_param_orders(leverage=leverage,side=side,size=size,
                        maintrade=False,sl_trade=False,tp_trade=False,close_position=True)
                        response,re,statusCode=self.request_order(parameters=listparams[0],meth='POST',num='1',addendpoint='',user=user)
                        print(8)
                        if statusCode==200:
                            print(2)
                            statusCode=0
                            response,re,statusCode=self.request_order(parameters='',meth='GET',num='13',addendpoint=self.symbol,user=user)

                            if (statusCode==200) and (len(response['data'])==0):
                                print(1)
                                self.alive_positions.pop(user)
                                response,re,statusCode=self.request_order(parameters='',meth='DELETE',num='3',addendpoint=self.symbol,user=user)
                                self.detail_accuonts[user]['status']=0
                                break

                    elif (statusCode==200) and (len(response['data'])==0):
                        print(3)
                        self.alive_positions.pop(user)
                        self.request_order(parameters='',meth='DELETE',num='3',addendpoint=self.symbol,user=user)
                        self.detail_accuonts[user]['status']=0
                        break

                    c+=1

            if side=='sell':
                print('sell-control_sl')
                while x==1:

                    response,re,statusCode=self.request_order(parameters='',meth='GET',num='13',addendpoint=self.symbol,user=user)
                    if c>=1:
                        listparams=self.set_param_orders(leverage=leverage,side=side,size=size,
                        maintrade=False,sl_trade=False,tp_trade=False,close_position=True)
                        response,re,statusCode=self.request_order(parameters=listparams[0],meth='POST',num='1',addendpoint='',user=user)

                        print(7)
                        if statusCode==200:
                            print(5)
                            statusCode=0
                            response,re,statusCode=self.request_order(parameters='',meth='GET',num='13',addendpoint=self.symbol,user=user)

                            if (statusCode==200) and (len(response['data'])==0):
                                print(4)
                                self.alive_positions.pop(user)
                                response,re,statusCode=self.request_order(parameters='',meth='DELETE',num='3',addendpoint=self.symbol,user=user)
                                self.detail_accuonts[user]['status']=0
                                break

                    elif (statusCode==200) and (len(response['data'])==0):
                        print(6)
                        self.alive_positions.pop(user)
                        self.request_order(parameters='',meth='DELETE',num='3',addendpoint=self.symbol,user=user)
                        self.detail_accuonts[user]['status']=0
                        break

                    c+=1

        return 

    def control_stoptrail(self,user,stopPrice_sl):

#         print('stop_trail= ',self.stop_trail , '  ' ,'stopPrice_sl= ',stopPrice_sl )
        size=self.detail_accuonts[user]['calculated_size']
        leverage=self.alive_positions[user]['leverage']
        side=self.alive_positions[user]['side']

        if side=='buy':
            print('buy-control_stoptrail')
            x=1
            while x==1:
                listparams=self.set_param_orders(leverage=leverage,side=side,size=size,
                stopPrice_sl=stopPrice_sl,maintrade=False,sl_trade=True,tp_trade=False)

                response,re,statusCode=self.request_order(parameters=listparams[0],meth='POST',num='1',addendpoint='',user=user)
                if statusCode==200:
                    self.detail_accuonts[user]['stopPrice_sl']=stopPrice_sl
                    break

        if side=='sell':
            print('sell-control_stoptrail')
            x=1
            while x==1:
                listparams=self.set_param_orders(leverage=leverage,side=side,size=size,
                stopPrice_sl=stopPrice_sl,maintrade=False,sl_trade=True,tp_trade=False)

                response,re,statusCode=self.request_order(parameters=listparams[0],meth='POST',num='1',addendpoint='',user=user)
                if statusCode==200:
                    self.detail_accuonts[user]['stopPrice_sl']=stopPrice_sl
                    break
        self.detail_accuonts[user]['trailprice']=self.price
        return

    def check_exist_alive_position(self,user,kwargs=None):

        x=1
        while x==1:
            response,re,statusCode=self.request_order(parameters='',meth='GET',num='13',addendpoint=self.symbol,user=user)
            if (len(response['data'])==0) and (statusCode==200):
                self.multi_threading(func=self.reset_users,accuonts=[*self.detail_accuonts])
                self.request_order(parameters='',meth='DELETE',num='3',addendpoint=self.symbol,user=user)
                print('seccful reset detail accuont')
                break
            elif (len(response['data'])!=0) and (statusCode==200):
                break

    def multi_threading(self,func,accuonts,**kwargs):

        threads=[]
        if len(kwargs)==0:
            kwargs=None
        for i in accuonts:
            user=i
            thread=threading.Thread(target=func,args=(user,kwargs,))
            thread.start()
            threads.append(thread)
        for thread in threads:
            thread.join()
        return

    def account_overview(self,user,kwargs=None):
        print('#####')

        print(self.detail_accuonts[user]['type_ballance'][1])
        response,re,statusCode=self.request_order(parameters='',meth='GET',num='1',addendpoint=self.detail_accuonts[user]['type_ballance'][1],user=user)
        self.detail_accuonts[user]['availableBalance']=response['data']['availableBalance']
        return

    def reset_users(self,user,kwargs=None):
        print('*****')

        zero=['availableBalance','calculated_size','id','leverage','side','size_position',
              'symbol','status','symbol','stopPrice_sl','stop_trail','trailprice','actionprice','sl','st',
              'status_sl','orderId_position','orderId_sl']
        for i in zero:
            self.detail_accuonts[user][i] = 0
        json.dump(self.detail_accuonts, open('detail_accuonts.json' , 'w'))
        return

    def get_signal(self):

        print('please wait for a signal')
        self.socket_message='signal'
        time.sleep(1)
        x=1
        while x==1:

            detail = self.socket_response.copy() # detail={'symbol':'' ,'balance':0-100% ,'st':'','leverage':'',
                                                                        #             'sl':'' ,'sellorbuy':None}

            if self.socket_response!=None:
                self.socket_response=None

                if (type(detail['st'])==float) and (type(detail['leverage'])==int) \
                and (type(detail["propertise_trader"]['sl'])==float) and (type(detail['sellorbuy'])==int):

                    if detail["propertise_trader"]['sl']>=0 and detail["propertise_trader"]['sl']<=0.05:
                        if (detail['sellorbuy']==1) or (detail['sellorbuy']==0):
                            for i in [*self.detail_accuonts]:

                                self.detail_accuonts[i]['st']=detail['st']
                                self.detail_accuonts[i]['leverage']=detail['leverage']
                                self.detail_accuonts[i]['sl']=detail["propertise_trader"]['sl']
                                json.dump(self.detail_accuonts, open('detail_accuonts.json' , 'w'))

                            self.symbol=detail["symbol"]
                            lot = json.load( open( "lot_size.json"))
                            self.one_lot=lot["multiplier"][self.symbol]

                            chime.theme('material')
                            chime.success()

                            break


        return detail

    def open_order(self):

        detail = self.get_signal()
        x=1
        while x==1:

            if self.price==0:
                time.sleep(5)
            else:
                break
        print('\n new order\n in ' ,detail,'\n')
        print('action price= ',self.price)
        print('self.exit_trader:',self.exit_trader)
        if self.exit_trader==1:
            for i in [*self.detail_accuonts]:
                self.detail_accuonts[i]['symbol']=self.symbol
            p=self.price
            threads_op=[]
            if detail['sellorbuy']==0:
                for i in [*self.detail_accuonts]:
                    self.detail_accuonts[i]['side']='sell'
                print('sell position')
                for i in [*self.detail_accuonts]:
                    self.detail_accuonts[i]['stopPrice_sl']=(1+(self.detail_accuonts[i]['sl']/self.detail_accuonts[i]['leverage']))\
                    *p - p*self.commission
                    self.detail_accuonts[i]['stop_trail']=(1-(self.detail_accuonts[i]['st']/self.detail_accuonts[i]['leverage']))\
                    *p - p*self.commission

                print(self.detail_accuonts)
                print(self.price)
#                 while x==1:
#                     x=1
                for i in [*self.detail_accuonts]:
                    thread=threading.Thread(target=self.open_position,args=(i,
                    self.detail_accuonts[i]['leverage'],self.detail_accuonts[i]['side'],round(self.detail_accuonts[i]['stopPrice_sl'],self.rond),))
                    thread.start()
                    threads_op.append(thread)
                for thread in threads_op:
                    thread.join()
                    threads_op.remove(thread)

            if detail['sellorbuy']==1:
                for i in [*self.detail_accuonts]:
                    self.detail_accuonts[i]['side']='buy'
                print('buy position')
                for i in [*self.detail_accuonts]:
                    self.detail_accuonts[i]['stopPrice_sl']=(1-(self.detail_accuonts[i]['sl']/self.detail_accuonts[i]['leverage']))\
                    *p + p*self.commission
                    self.detail_accuonts[i]['stop_trail']=(1+(self.detail_accuonts[i]['st']/self.detail_accuonts[i]['leverage']))\
                    *p + p*self.commission
                    
                print(self.detail_accuonts)
                print(self.price)
#                 while x==1:
#                     x=1
                for i in [*self.detail_accuonts]:
                    thread=threading.Thread(target=self.open_position,args=(i,
                    self.detail_accuonts[i]['leverage'],self.detail_accuonts[i]['side'],round(self.detail_accuonts[i]['stopPrice_sl'],self.rond),))
                    thread.start()
                    threads_op.append(thread)
                for thread in threads_op:
                    thread.join()
                    threads_op.remove(thread)
        return

    def control_position_buy(self):
        self.socket_message='control'
        
        self.touched_st=[]
        self.touched_sl=[]
        
        print('control buy')
        
        x=1
        while x==1:
            
            sp = self.socket_response.copy() #{'stop_trail':0,'stopPrice_sl':0}
            if self.socket_response!=None:
                self.socket_response=None

#                 if (type(sp['stop_trail'])==int) and (type(sp['stopPrice_sl'])==int):
#                     if (sp['stop_trail']>0) or (sp['stopPrice_sl']>0):
                        #مورد بالا را در قسمت تریدر کنترل کنم و پیام بدم بهش اگه اشتباه بود
                    self.sl_tp=sp.copy()
                    
            for i in [*self.alive_positions]:
                self.detail_accuonts[i]['last_time']=str(datetime.datetime.utcnow())
            for i in [*self.alive_positions]:
                self.alive_positions[i]['stop_trail']=(1+(self.alive_positions[i]['st']/self.alive_positions[i]['leverage']))\
                *self.alive_positions[i]['trailprice'] + self.alive_positions[i]['actionprice']*self.commission

            self.touched_sl=[*{key:value for key,value in self.alive_positions.items() if value['stopPrice_sl']>=self.price}]
            self.touched_st=[*{key:value for key,value in self.alive_positions.items() if value['stop_trail']<=self.price}]

            if (len(self.touched_sl)>0) or (self.sl_tp['stopPrice_sl']>0):
                self.sl_tp={'stop_trail':0,'stopPrice_sl':0}

                self.touched_sl=[*self.alive_positions]
                print('touch sl')
                
                for i in self.touched_sl:
                    self.alive_positions[i]['lastprice']=self.price
                    self.alive_positions[i]['PNL']= ((self.alive_positions[i]['lastprice']-self.alive_positions[i]['actionprice'])\
                    /self.alive_positions[i]['actionprice'])*100
                try:
                    trads=json.load(open("trads.json"))
                except Exception as error:
                    trads={}
                    json.dump(trads, open('trads.json', 'w'))

                if self.username in trads:
                    l=len(trads[self.username])
                    trads[self.username]['%s'%(l+1)]=self.alive_positions
                else:
                    trads[self.username]={}
                    l=len(trads[self.username])
                    trads[self.username]['%s'%(l+1)]=self.alive_positions
                json.dump(trads, open('trads.json', 'w'))

#                 print(self.alive_positions)
#                 break

                for i in self.touched_sl:
                    self.touched_sl.remove(i)
            
                    self.control_sl(i,self.alive_positions[i]['leverage'],self.alive_positions[i]['side'])
  
                print(12)

            elif len([*self.alive_positions])==0:
                print(16)
                break

            elif (len(self.touched_st)>0) or (self.sl_tp['stop_trail']>0):
                self.sl_tp={'stop_trail':0,'stopPrice_sl':0}


                self.touched_st=[*self.alive_positions]
                print('touch st')
                for i in self.touched_st:
                    self.alive_positions[i]['stopPrice_sl']=(1-(self.alive_positions[i]['sl']/self.alive_positions[i]['leverage']))\
                    *self.price + self.alive_positions[i]['actionprice']*self.commission
                
                for i in self.touched_st:
                    self.touched_st.remove(i)

                    self.control_stoptrail(i,round(self.alive_positions[i]['stopPrice_sl'],self.rond))
                

                self.alive_positions={key:value for key,value in self.detail_accuonts.items() if value['status']==1}
                json.dump(self.detail_accuonts, open('detail_accuonts.json' , 'w'))


        return

    def control_position_sell(self):
        
        self.touched_st=[]
        self.touched_sl=[]
        

        print('control sell')
        x=1
        while x==1:
            
            sp = self.socket_response.copy() #{'stop_trail':0,'stopPrice_sl':0}
            if self.socket_response!=None:
                self.socket_response=None
                
#             if (type(sp['stop_trail'])==int) and (type(sp['stopPrice_sl'])==int):
#                 if (sp['stop_trail']>0) or (sp['stopPrice_sl']>0):
                        #مورد بالا را در قسمت تریدر کنترل کنم و پیام بدم بهش اگه اشتباه بود
                    
                self.sl_tp=sp
    
            for i in [*self.alive_positions]:
                self.detail_accuonts[i]['last_time']=str(datetime.datetime.utcnow())
            
            for i in [*self.alive_positions]:
                self.alive_positions[i]['stop_trail']=(1-(self.alive_positions[i]['st']/self.alive_positions[i]['leverage']))\
                *self.alive_positions[i]['trailprice'] - self.alive_positions[i]['actionprice']*self.commission

            self.touched_sl=[*{key:value for key,value in self.alive_positions.items() if value['stopPrice_sl']<=self.price}]
            self.touched_st=[*{key:value for key,value in self.alive_positions.items() if value['stop_trail']>=self.price}]

            if (len(self.touched_sl)>0) or (self.sl_tp['stopPrice_sl']>0):
                self.sl_tp={'stop_trail':0,'stopPrice_sl':0}

                print('touch sl')

                self.touched_sl=[*self.alive_positions]

                for i in self.touched_sl:
                    self.alive_positions[i]['lastprice']=self.price
                    self.alive_positions[i]['PNL']= ((self.alive_positions[i]['actionprice']-self.alive_positions[i]['lastprice'])\
                    /self.alive_positions[i]['actionprice'])*100
                try:
                    trads=json.load(open("trads.json"))
                except Exception as error:
                    trads={}
                    json.dump(trads, open('trads.json', 'w'))

                if self.username in trads:
                    l=len(trads[self.username])
                    trads[self.username]['%s'%(l+1)]=self.alive_positions
                else:
                    trads[self.username]={}
                    l=len(trads[self.username])
                    trads[self.username]['%s'%(l+1)]=self.alive_positions
                json.dump(trads, open('trads.json', 'w'))


#                 print(self.alive_positions)
#                 break

                for i in self.touched_sl:
                    self.touched_sl.remove(i)
                    user=i
                    self.control_sl(i,self.alive_positions[i]['leverage'],self.alive_positions[i]['side'])

                print(19)

            elif len([*self.alive_positions])==0:
                print(20)
                break

            elif (len(self.touched_st)>0) or (self.sl_tp['stop_trail']>0):
                self.sl_tp={'stop_trail':0,'stopPrice_sl':0}

                print('touch st')
                self.touched_st=[*self.alive_positions]
                for i in self.touched_st:
                    self.alive_positions[i]['stopPrice_sl']=(1+(self.alive_positions[i]['sl']/self.alive_positions[i]['leverage']))\
                    *self.price - self.alive_positions[i]['actionprice']*self.commission

                for i in self.touched_st:
                    self.touched_st.remove(i)
                    
                    self.control_stoptrail(i,round(self.alive_positions[i]['stopPrice_sl'],self.rond))


                self.alive_positions={key:value for key,value in self.detail_accuonts.items() if value['status']==1}
                json.dump(self.detail_accuonts, open('detail_accuonts.json' , 'w'))

                print(23)
        return

    def control_position(self):
        
        if self.exit_trader==1:
            x=1
            self.detail_accuonts=self.creat_accuont()
            display(self.detail_accuonts)

            thread=threading.Thread(target=self.connection)
            thread.start()

            thread=threading.Thread(target=self.runprice)
            thread.start()

            thread=threading.Thread(target=self.refresh_runprice)
            thread.start()
        else:
            x=0

        while x==1:
            if self.exit_trader==0:
                x=0
    # -----------------------------------------------------------------------
            
            if (len([*{key:value for key,value in self.detail_accuonts.items() if value['status']>0}])==0)\
            and (self.exit_trader==1): 
                self.multi_threading(func=self.account_overview,accuonts=[*self.detail_accuonts])

    # -----------------------------------------------------------------------
            # open position

            if (len([*{key:value for key,value in self.detail_accuonts.items() if value['status']>0}])==0)\
            and (self.exit_trader==1):
                self.open_order()
    # -----------------------------------------------------------------------
            #control position

            if (len([*{key:value for key,value in self.detail_accuonts.items() if value['status']>0}])>0):
                print ('start control position')
                self.alive_positions={key:value for key,value in self.detail_accuonts.items() if value['status']>0}

                for i in [*self.alive_positions]:
                    if self.alive_positions[i]['side']=='buy':
                        side='buy'
                        break
                    elif self.alive_positions[i]['side']=='sell':
                        side='sell'
                        break


                while x==1:
                    if self.price==0:
                        time.sleep(5)
                    else:
                        break
                if side=='buy':
                    self.control_position_buy()

                elif side=='sell':
                    self.control_position_sell()

    # -----------------------------------------------------------------------
            self.multi_threading(func=self.check_exist_alive_position,accuonts=[*self.detail_accuonts])

        return
    
try:
    user=json.load(open('users.json'))
except Exception as error:
    pass
one_lot=10 # xrp
symbol='XRPUSDTM'
timeframe='5m'
limit=540
commission=0.001
rond=4
trade=trader( rond=rond , commission=commission , username=[*user][0])

trade.control_position()
# پیامها را وقتی با سوکت میفرستم باید اسم یوزر را به آن اضافه کنم و همینطور شناسه ای پی آی کاربر و توکن آن را و اینها باید در تابع کانکشن انجام شود
# من باید از طریق سوکت پیامها را برای کنترل پوزیشن و گرفتن سیگنال استفاده کنم

In [ ]:
import socket
import threading
import sqlite3
import datetime
x=1
host = '127.0.0.1'
port = 10000

ClientSocket = socket.socket()
print('Waiting for connection')
try:
    ClientSocket.connect((host, port))
except socket.error as e:
    print(str(e))
Response = ClientSocket.recv(2048).decode('utf-8')
print(Response)
while x==1:
    Input = input('Enter your name:')

    ClientSocket.send(str.encode(Input))
    socket_response = ClientSocket.recv(2048).decode('utf-8')
    print(socket_response)
ClientSocket.close()

Waiting for connection
you are conected now
Enter your name:hg
you are conected now eee
